In [43]:
import xport
import pandas as pd
import json

# Notebook to Process NHANE Food Interview Data 
To find pie of course. 

You will need to download the raw files in XPT format from [CDC and USDA NHANES Pages](https://www.cdc.gov/nchs/nhanes/wweia.htm)

In [2]:
def get_data(fname):
    """ 
    Place all the data in the file `fname` into a dictionary indexed 
    by sequence number.

    Arguments:
    ----------
    fname : The file name of the data

    Returns:
    --------
    Z : A dictionary mapping the sequence numbers to lists of data values
    H : The names of the variables, in the order they appear in Z
    CREDIT: http://dept.stat.lsa.umich.edu/~kshedden/Python-Workshop/nhanes_data.html
    """

    ## The data, indexed by sequence number
    Z = {}

    ## The variable names, in the order they appear in values of Z.
    H = None

    with open(fname, 'rb') as f:
        for row in xport.DictReader(f):

            ## Get the variable names from the first record
            if H is None:
                H = list(row.keys())
                H.remove("SEQN")
                H.sort()

            Z[''.join([str(row["SEQN"]), str(row["DR1ILINE"])])] = {k:row[k] for k in H} #modified to allow for food lines

    return Z,H

In [3]:
r, h = get_data('DR1IFF_H.XPT')

In [25]:
pie_codes = {}
with open('drxfcd_h.xpt', 'rb') as f:
        for row in xport.DictReader(f):
            if 'Pie,' in  row['DRXFCLD']:
                pie_codes[row['DRXFDCD']] = row['DRXFCLD']
            
len(pie_codes)
            
            
            

90

In [19]:
all_data = pd.DataFrame.from_dict(r, orient='index')

In [20]:
pie_only = all_data[all_data.DR1IFDCD.isin(pie_codes)]

In [21]:
interesting_columns = ['DR1_030Z','DR1_040Z','DR1IKCAL', 'DR1FS','DR1IFDCD']
pie_only = pie_only[interesting_columns].rename(columns={'DR1_030Z':'EatingOccasion','DR1_040Z':'AtHome'
                                               ,'DR1IKCAL':'Calories'
                                               ,'DR1FS':'Source', 'DR1IFDCD':'FoodCode'})





In [24]:
pie_only.shape

(164, 5)

In [35]:
def EatingOccasion(val):
    int_val = int(val)
    eating_occasions ={1:'Breakfast',2:'Lunch',3:'Dinner',4:'Dinner',5:'Brunch',6:'Snack',7:'Drink', 13:'Meirenda',14:'Cena',15:'Entre comida'}
    return eating_occasions[int_val]

def atHome(val):
    int_val = int(val)
    atHome_vals = {1:True,2:False}
    return atHome_vals[int_val]


def Source(val):
    int_val = int(val)
    source_vals = {1:'Store', 2:'Restaurant with Waiter/Waitress', 3:'FastFood',
                   4:'Bar/Tavern',6:'Non-School Cafeteria',7:'School Cafeteria',8:'Care Center',10:'Soup Kitchen or Food Pantry'
                   ,11:'Meals on Wheels',14:'Vending Machine',16:'From Somewhere Else/Gift', 19:'Grown by Someone you know'
                   , 17:'Mail order purchase', 24:'Sports Facility',25:'Street Vendor'
                   ,27:'Convenience Store',26:'Bake Sale', 99:'Other'}
    return source_vals[int_val]

def foodCode(val):
    words = pie_codes[val].split(',')
    flavor = words[1]
    return flavor

def Mapper(row):
    new_row = {'Source':Source(row['Source']), 'EatingOccasion':EatingOccasion(row['EatingOccasion'])
               , 'AtHome':atHome(row['AtHome']), 'Calories':row['Calories'], 'FoodCode':foodCode(row['FoodCode'])}
    return new_row

                  
                  
                  
pie_output = pd.DataFrame.from_dict(pie_only.apply(Mapper, axis=1).to_dict(), orient='index')    

In [53]:
pie_output.to_csv('Pie_Data.csv')